In [3]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Read the CSV file
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Identify target and feature variables
target = df["IS_SUCCESSFUL"]
features = df.drop(columns=["EIN", "NAME", "IS_SUCCESSFUL"])

# Determine unique values for each column
unique_counts = features.nunique()

# Adjust for any modifications based on model optimization
cutoff = 100
for column in unique_counts.index:
    if unique_counts[column] > 10 and features[column].dtype == "object":
        counts = features[column].value_counts()
        rare_values = counts[counts < cutoff].index
        features[column] = features[column].apply(lambda x: "Other" if x in rare_values else x)

# Encode variables using pd.get_dummies()
features_encoded = pd.get_dummies(features)

# Split the preprocessed data into features array (X) and target array (y)
X = features_encoded.values
y = target.values

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Scale the training and testing features datasets using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [5]:
# Create a neural network
model = Sequential()

# Add the first hidden layer w/ more neurons
model.add(Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]))

# Add a second hidden layer w/ more neurons (optional)
model.add(Dense(units=64, activation='relu'))

# Add a third hidden layer w/ more neurons (optional)
model.add(Dense(units=32, activation='relu'))

# Add output layer
model.add(Dense(units=1, activation='sigmoid'))

# Check structure of the model
model.summary()

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create a callback to save the model's weights every five epochs
checkpoint = ModelCheckpoint("/content/drive/MyDrive/path_to_your/model_weights_optimized.h5",
                             save_weights_only=True, save_best_only=True,
                             monitor='val_loss', mode='min', verbose=1, period=5)

# Train the model with increased epochs
history = model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    batch_size=256,
    validation_data=(X_test_scaled, y_test),
    callbacks=[checkpoint]
)

# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Save the model to an HDF5 file
model.save("/content/drive/MyDrive/path_to_your/AlphabetSoupCharity_Optimization.h5")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               6528      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 16897 (66.00 KB)
Trainable params: 16897 (66.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/100
101/101 [==============================] - 2s 9ms/step - loss: 0.5781 - accuracy: 0.7097 - val_loss: 0.5586 - val_accuracy: 0.7310
Epoch 2/100
101/101 [==============================] - 1s 8ms/step - loss: 0.5523 - accuracy: 0.7280 - val_loss: 0.5539 - val_accuracy: 0.7226
Epoch 3/100
101/101 [==============================] - 1s 7ms/step - loss: 0.5485 - accuracy: 0.7301 - val_loss: 0.5512 - val_accuracy: 0.7307
Epoch 4/100
101/101 [==============================] - 1s 8ms/step - loss: 0.5461 - accuracy: 0.7303 - val_loss: 0.5514 - val_accuracy: 0.7270
Epoch 5/100
 96/101 [===========================>..] - ETA: 0s - loss: 0.5455 - accuracy: 0.7331
Epoch 5: val_loss improved from inf to 0.55203, saving model to /content/drive/MyDrive/path_to_your/model_weights_optimized.h5
101/101 [==============================] - 1s 9ms/step - loss: 0.5450 - accuracy: 0.7334 - val_loss: 0.5520 - val_accuracy: 0.7273
Epoch 6/100
101/101 [==============================] - 0s 5ms/step - loss

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
